<a href="https://colab.research.google.com/github/mfmarlonferrari/NietzscheLLM/blob/main/IntroLLMTransformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tiktoken
!pip install transformers[torch]
!pip install pandas
!pip instal numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 3.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux

In [ ]:
import numpy as np
import pandas as pd
import tiktoken
from transformers import RobertaConfig
from transformers import RobertaTokenizer
from transformers import RobertaForMaskedLM
from transformers import pipeline
from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from tokenizers import ByteLevelBPETokenizer
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import ByteLevel

In [ ]:

# Gerar dados de treinamento e validação
np.random.seed(42)
train_data = [(np.random.randint(0, 1000), np.random.randint(0, 1000)) for _ in range(10000)]
val_data = [(np.random.randint(0, 1000), np.random.randint(0, 1000)) for _ in range(2000)]

# Preparar os dados em um DataFrame

#df para treinar
train_df = pd.DataFrame(train_data, columns=['num1', 'num2'])
train_df['sum'] = train_df['num1'] + train_df['num2']


frase = []
for i in train_df.index:
    x = train_df.iloc[i].values
    frase.append(f'{x[0]} + {x[1]} = {x[2]}')

In [ ]:
len(frase)

10000

In [ ]:
# Salva os dados de treinamento em um arquivo temporário
with open("temp_dados_treino.txt", "w") as f:
    for item in frase:
        f.write(f"{item}\n")

In [ ]:
# Inicializa o ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer()

# Treina o tokenizador
tokenizer.train(files=["temp_dados_treino.txt"], vocab_size=10_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

In [ ]:
# Salva o tokenizador treinado
!mkdir modelo_tokenizer
tokenizer.save_model("modelo_tokenizer")

['modelo_tokenizer/vocab.json', 'modelo_tokenizer/merges.txt']

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('modelo_tokenizer', max_len=512)

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=10_000,
    max_position_embeddings=312,
    num_attention_heads=6,
    num_hidden_layers=3,
    type_vocab_size=1,
)

from transformers import RobertaForMaskedLM
model = RobertaForMaskedLM(config=config)

In [ ]:
model.num_parameters()

29787664

In [ ]:
#forma simples de se carregar um arquivo bruto como Dataset
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path='temp_dados_treino.txt',
    block_size=128,
)

In [ ]:
# verificando
dataset.examples[:2]

[{'input_ids': tensor([   0, 1636,  261, 1205,  262,  678,    2])},
 {'input_ids': tensor([   0, 2589,  261,  762,  262, 2175,    2])}]

In [ ]:
tokenizer.decode(dataset.examples[7]['input_ids'])

'<s>87 + 372 = 459</s>'

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.1
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='modelo_tokenizer',
    overwrite_output_dir=True,
    num_train_epochs=100,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
trainer.train()

Step,Training Loss
500,5.077100
1000,4.567600
1500,4.399300
2000,4.320600
2500,4.313800
3000,4.149200
3500,4.050900
4000,3.928100
4500,3.904800
5000,3.813200


TrainOutput(global_step=15700, training_loss=3.5351110062325835, metrics={'train_runtime': 27692.8902, 'train_samples_per_second': 36.11, 'train_steps_per_second': 0.567, 'total_flos': 981285967902720.0, 'train_loss': 3.5351110062325835, 'epoch': 100.0})

In [ ]:
trainer.save_model('modelo_tokenizer')

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model='modelo_tokenizer',
    tokenizer='modelo_tokenizer'
)

In [ ]:
def retornaMask(lista):
    maior = 0
    resulta = ''
    for i in resultado:
        if i['score'] > maior:
            maior = i['score']
            resulta = i['token_str']
    return resulta

In [ ]:
resultado = fill_mask('1 + <mask> = 10')
resultado_final = retornaMask(resultado)

print("O numero a ser soma é o:", resultado_final)

In [ ]:
resultado = fill_mask('55 + <mask> = 188')
resultado_final = retornaMask(resultado)

print("O numero a ser soma é o:", resultado_final)

O numero a ser soma é o:  54


In [ ]:
resultado = fill_mask('<mask> + 9 = 10')
resultado_final = retornaMask(resultado)

print("O numero a ser soma é o:", resultado_final)

O numero a ser soma é o: 1


In [ ]:
resultado = fill_mask('<mask> + 90 = 100')
resultado_final = retornaMask(resultado)

print("O numero a ser soma é o:", resultado_final)

O numero a ser soma é o: 11


In [ ]:
resultado = fill_mask('100 + 900 = <mask>')
resultado_final = retornaMask(resultado)

print("O resultado da soma é:", resultado_final)

O resultado da soma é:  797


In [ ]:
resultado = fill_mask('150 + 900 = <mask>')
resultado_final = retornaMask(resultado)

print("O resultado da soma é:", resultado_final)

O resultado da soma é:  1135
